<a href="https://colab.research.google.com/github/BrokenShell/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-01-29 03:16:41--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.1s    

2020-01-29 03:16:46 (132 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [0]:
from scipy.stats import ttest_ind
import pandas as pd


column_headers = [
    'party','handicapped-infants','water-project', 'budget','physician-fee-freeze', 
    'el-salvador-aid', 'religious-groups','anti-satellite-ban', 'aid-to-contras',
    'mx-missile','immigration', 'synfuels', 'education', 'right-to-sue','crime',
    'duty-free', 'south-africa',
]
df = pd.read_csv(
    'house-votes-84.data', 
    header=None, 
    names=column_headers,
    na_values="?",
)
df = df.replace({'y': 1, 'n': 0})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


### Splitting Data

In [0]:
rep = df[df['party']=='republican']
dem = df[df['party']=='democrat']

## Stretch Goal 1: Refactored Functions

In [0]:
def custom_ttest(alpha, beta, col):
    statistic, pvalue = ttest_ind(alpha[col], beta[col], nan_policy='omit')
    return statistic, pvalue


def quick_stats(alpha, beta, cols):
    return '\n'.join(
        f"{col}: {custom_ttest(alpha, beta, col)}" for col in cols
    )

In [0]:
def detailed_output(title, col_name):
    print(f'{title} Ttest')
    statistic, pvalue = custom_ttest(rep, dem, col_name)
    print(f"Stat: {statistic}, Pvalue: {pvalue}")
    print(f'Rep: {rep[col_name].mean()*100:.02f}%')
    print(f'Dem: {dem[col_name].mean()*100:.02f}%')

In [0]:
# Quickly looking at all the issues:
print(quick_stats(rep, dem, column_headers[1:]))

handicapped-infants: (-9.205264294809222, 1.613440327937243e-18)
water-project: (0.08896538137868286, 0.9291556823993485)
budget: (-23.21277691701378, 2.0703402795404463e-77)
physician-fee-freeze: (49.36708157301406, 1.994262314074344e-177)
el-salvador-aid: (21.13669261173219, 5.600520111729011e-68)
religious-groups: (9.737575825219457, 2.3936722520597287e-20)
anti-satellite-ban: (-12.526187929077842, 8.521033017443867e-31)
aid-to-contras: (-18.052093200819733, 2.82471841372357e-54)
mx-missile: (-16.437503268542994, 5.03079265310811e-47)
immigration: (1.7359117329695164, 0.08330248490425066)
synfuels: (-8.293603989407588, 1.5759322301054064e-15)
education: (20.500685724563073, 1.8834203990450192e-64)
right-to-sue: (13.51064251060933, 1.2278581709672758e-34)
crime: (16.342085656197696, 9.952342705606092e-47)
duty-free: (-12.853146132542978, 5.997697174347365e-32)
south-africa: (-6.849454815841208, 3.652674361672226e-11)


## The Null Hypothesis for each of the following: Rep and Dem vote the same.

### Budget
Higher Dem support

In [0]:
detailed_output('Budget Support', 'budget')

Budget Support Ttest
Stat: -23.21277691701378, Pvalue: 2.0703402795404463e-77
Rep: 13.41%
Dem: 88.85%


Rejecting the null hypothesis for Budget

### Physician Fee Freeze
Higher Rep support

In [0]:
detailed_output('Physician Fee Freeze Support', 'physician-fee-freeze')

Physician Fee Freeze Support Ttest
Stat: 49.36708157301406, Pvalue: 1.994262314074344e-177
Rep: 98.79%
Dem: 5.41%


Rejecting the null hypothesis for Physician Fee Freeze

### Water Project
Nearly equal support

In [0]:
detailed_output('Water Project Support', 'water-project')

Water Project Support Ttest
Stat: 0.08896538137868286, Pvalue: 0.9291556823993485
Rep: 50.68%
Dem: 50.21%


Failing to reject the null hypothesis for the Water Project

## Stretch Goal 2: Personal Project

# Monster Data
### CR vs. Other Stats
Null Hypothesis: Stats are random and not correlated to the CR.

In [0]:
monsters = pd.read_csv('https://sharpdesigndigital.com/monsters.csv')

In [0]:
high_level = monsters[monsters['CR'] > 15]
low_level = monsters[monsters['CR'] < 16]

In [0]:
print(quick_stats(low_level, high_level, list(monsters.columns[1:])))

CR: (-87.12297993966718, 0.0)
AC: (-54.95820265444433, 0.0)
HP: (-88.06273482232656, 0.0)
ATT: (-62.31378999391324, 0.0)
DAM: (-91.52281727474525, 0.0)
Tier: (-88.35142405083205, 0.0)


Rejecting the Null hypothesis across the board.